# This Script analyzes events.csv

In [1]:
# Importing the libraries
import os
from matplotlib import pyplot as plt
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import *
import numpy as np
from pyspark.sql import SQLContext
from pyspark import SparkContext

In [2]:
clTr = "gs://jupyterbucket/outbrainData/clicks_train.csv"
evt = "gs://jupyterbucket/outbrainData/events.csv"
proCon = "gs://jupyterbucket/outbrainData/promoted_content.csv"

## Overview

In [3]:
def csvOverview(fpath):
    '''
    Function presents a basic overview of the file fed in the argument
    @params
    fpath: Path to the csv file that needs to be analyzed
    @returns 
    None
    '''
    # Reading the data as a spark dataframe
    print fpath.split("/")[-1]
    fpathDF = spark.read.options(header='true', inferschema='true', nullValue='\\N') \
                .csv(fpath)
    print "Dataframe has ", fpathDF.count(), " rows."
    fpathDF.show(n=5)
    
    return None

### Taking an overview of the data

In [4]:
csvOverview(evt)

events.csv
Dataframe has  23120126  rows.
+----------+--------------+-----------+---------+--------+------------+
|display_id|          uuid|document_id|timestamp|platform|geo_location|
+----------+--------------+-----------+---------+--------+------------+
|         1|cb8c55702adb93|     379743|       61|       3|   US>SC>519|
|         2|79a85fa78311b9|    1794259|       81|       2|   US>CA>807|
|         3|822932ce3d8757|    1179111|      182|       2|   US>MI>505|
|         4|85281d0a49f7ac|    1777797|      234|       2|   US>WV>564|
|         5|8d0daef4bf5b56|     252458|      338|       2|       SG>00|
+----------+--------------+-----------+---------+--------+------------+
only showing top 5 rows



#### Lets now see how many unique display_ids, uuid, document_ids, timestamp, platform and geo_locations are there. As mentioned above, we have total 23,120,126 datapoints.

In [5]:
evtSchema = StructType(
                    [StructField("display_id", StringType(), True),
                     StructField("uuid", StringType(), True),
                     StructField("document_id", StringType(), True),
                     StructField("timestamp", IntegerType(), True),
                     StructField("platform", StringType(), True),
                     StructField("geo_location", StringType(), True)])

In [6]:
evtDF = spark.read.schema(evtSchema).options(header='true', inferschema='false', nullValue='\\N').csv(evt)
evtDF.cache()

DataFrame[display_id: string, uuid: string, document_id: string, timestamp: int, platform: string, geo_location: string]

In [7]:
print "Number of unique display_ids: ", evtDF.select("display_id").distinct().count()
print "Number of unique uuids: ", evtDF.select("uuid").distinct().count()
print "Number of unique document_ids: ", evtDF.select("document_id").distinct().count()
print "Number of unique platforms: ", evtDF.select("platform").distinct().count()
print "Number of unique geo_locations: ", evtDF.select("geo_location").distinct().count()

Number of unique display_ids:  23120126
Number of unique uuids:  19794967
Number of unique document_ids:  894060
Number of unique platforms:  4
Number of unique geo_locations:  2989


### We can merge these ad_ids with the training set

In [8]:
csvOverview(clTr)

clicks_train.csv
Dataframe has  87141731  rows.
+----------+------+-------+
|display_id| ad_id|clicked|
+----------+------+-------+
|         1| 42337|      0|
|         1|139684|      0|
|         1|144739|      1|
|         1|156824|      0|
|         1|279295|      0|
+----------+------+-------+
only showing top 5 rows



In [9]:
clTrSchema = StructType(
                    [StructField("display_id", StringType(), True),
                     StructField("ad_id", StringType(), True),
                     StructField("clicked", IntegerType(), True)])

In [10]:
clTrDF = spark.read.schema(clTrSchema).options(header='true', inferschema='false', nullValue='\\N').csv(clTr)
clTrDF.cache()
clTrDF.show(n=7)

+----------+------+-------+
|display_id| ad_id|clicked|
+----------+------+-------+
|         1| 42337|      0|
|         1|139684|      0|
|         1|144739|      1|
|         1|156824|      0|
|         1|279295|      0|
|         1|296965|      0|
|         2|125211|      0|
+----------+------+-------+
only showing top 7 rows



In [11]:
print "Number of unique display_ids: ", clTrDF.select("display_id").distinct().count()
print "Number of unique ad_ids: ", clTrDF.select("ad_id").distinct().count()

Number of unique display_ids:  16874593
Number of unique ad_ids:  478950


### Merging the events.csv with clicks_train.csv

In [12]:
clTrevtDF = clTrDF.join(evtDF, on="display_id", how="left")
clTrevtDF.cache()
clTrevtDF.show(n=15)

+----------+------+-------+--------------+-----------+---------+--------+------------+
|display_id| ad_id|clicked|          uuid|document_id|timestamp|platform|geo_location|
+----------+------+-------+--------------+-----------+---------+--------+------------+
|  10000108|132815|      1|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807|
|  10000108|133677|      0|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807|
|  10000108|406686|      0|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807|
|  10000108|406704|      0|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807|
|  10000108|449087|      0|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807|
|  10000108|471551|      0|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807|
|  10000108|488469|      0|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807|
|  10000172| 45192|      0|11c89d2bcfd60b|    1233989|668711251|       1|   US>OK>671|
|  10000172|106790|      0|11c89d2bcfd60b| 

#### Now looking at promoted_content.csv

In [13]:
csvOverview(proCon)

promoted_content.csv
Dataframe has  559583  rows.
+-----+-----------+-----------+-------------+
|ad_id|document_id|campaign_id|advertiser_id|
+-----+-----------+-----------+-------------+
|    1|       6614|          1|            7|
|    2|     471467|          2|            7|
|    3|       7692|          3|            7|
|    4|     471471|          2|            7|
|    5|     471472|          2|            7|
+-----+-----------+-----------+-------------+
only showing top 5 rows



In [14]:
proConSchema = StructType(
                    [StructField("ad_id", StringType(), True),
                     StructField("document_id", StringType(), True),
                     StructField("campaign_id", StringType(), True),
                     StructField("advertiser_id", IntegerType(), True)])

In [15]:
proConDF = spark.read.schema(proConSchema).options(header='true', inferschema='false', nullValue='\\N').csv(proCon)
proConDF.cache()
proConDF.show(n=7)

+-----+-----------+-----------+-------------+
|ad_id|document_id|campaign_id|advertiser_id|
+-----+-----------+-----------+-------------+
|    1|       6614|          1|            7|
|    2|     471467|          2|            7|
|    3|       7692|          3|            7|
|    4|     471471|          2|            7|
|    5|     471472|          2|            7|
|    6|      12736|          1|            7|
|    7|      12808|          1|            7|
+-----+-----------+-----------+-------------+
only showing top 7 rows



#### In the above dataframe, it must be noted that the document_id mentioned here is the document where clicking on the ad takes the user and not the document_id on which the ad occurs. So we need to rename the column to ad_document_id

In [27]:
proConRenDF = proConDF.withColumnRenamed("document_id", "adDoc_id").withColumnRenamed("campaign_id", "adCamp_id").withColumnRenamed("advertiser_id", "advert_id")
proConRenDF.cache()
proConRenDF.show(n=7)

+-----+--------+---------+---------+
|ad_id|adDoc_id|adCamp_id|advert_id|
+-----+--------+---------+---------+
|    1|    6614|        1|        7|
|    2|  471467|        2|        7|
|    3|    7692|        3|        7|
|    4|  471471|        2|        7|
|    5|  471472|        2|        7|
|    6|   12736|        1|        7|
|    7|   12808|        1|        7|
+-----+--------+---------+---------+
only showing top 7 rows



In [29]:
print "Number of unique ad_ids: ", proConRenDF.select("ad_id").distinct().count()
print "Number of unique adDocument_ids: ", proConRenDF.select("adDoc_id").distinct().count()
print "Number of unique adCampaign_ids: ", proConRenDF.select("adCamp_id").distinct().count()
print "Number of unique advertiser_id: ", proConRenDF.select("advert_id").distinct().count()

 Number of unique ad_ids:  559583
Number of unique adDocument_ids:  185709
Number of unique adCampaign_ids:  34675
Number of unique advertiser_id:  4385


#### Now we merge the information on the ad with the previoulsy merged data

In [30]:
clTrevtproConRenDF = clTrevtDF.join(proConRenDF, on="ad_id", how="left")
clTrevtproConRenDF.cache()
clTrevtproConRenDF.show()

+------+----------+-------+--------------+-----------+----------+--------+------------+--------+---------+---------+
| ad_id|display_id|clicked|          uuid|document_id| timestamp|platform|geo_location|adDoc_id|adCamp_id|advert_id|
+------+----------+-------+--------------+-----------+----------+--------+------------+--------+---------+---------+
|100010|  15514599|      0|c306657bf72e87|    1914050|1029468082|       1|          US| 1132761|    13049|     2848|
|100010|  15434742|      0|7cf0aa77cad72d|     339764|1023601796|       1|   US>FL>539| 1132761|    13049|     2848|
|100227|   5383514|      0|99d4cc8820a854|    1703188| 349564881|       1|   US>OR>820| 1140509|    13071|     2848|
|100227|   5671562|      0|f9a74edb4874db|     313715| 378329714|       1|   US>MO>609| 1140509|    13071|     2848|
|100227|   5724166|      0|54594b6166b827|     708515| 381430184|       3|   US>OR>820| 1140509|    13071|     2848|
|100227|   9023708|      0|43ceee3456b0b4|    2299068| 605463584

In [32]:
clTrevtproConRenDF[clTrevtproConRenDF.display_id=="10000108"].show()

+------+----------+-------+--------------+-----------+---------+--------+------------+--------+---------+---------+
| ad_id|display_id|clicked|          uuid|document_id|timestamp|platform|geo_location|adDoc_id|adCamp_id|advert_id|
+------+----------+-------+--------------+-----------+---------+--------+------------+--------+---------+---------+
|406686|  10000108|      0|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807| 1671134|    27505|     2879|
|132815|  10000108|      1|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807| 1227645|    17018|      331|
|406704|  10000108|      0|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807| 1678714|    27506|     2879|
|488469|  10000108|      0|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807| 2387648|       65|       29|
|133677|  10000108|      0|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807| 1297868|    17143|     1919|
|449087|  10000108|      0|66d17a9ccd0e15|    1126487|668707895|       1